In [1]:
import numpy as np
import cv2
import imutils
import PIL
from PIL import Image, ImageDraw, ImageFilter
from argparse import ArgumentParser
import glob

# parse variables

In [ ]:
def parse_args():
    """Command-line argument for rain synthesis"""

    # New parser
    parser = ArgumentParser(description='Rain Synthesis')

    # Data parameters
    parser.add_argument('-ks', '--kernel-size',
                        help='kenel size of motion blur, can be used to increase rain streak size', default='30')
    parser.add_argument('-dl', '--drop-length', help='rain streak length. best to keep it small', default=20, type=int)
    parser.add_argument('-dw', '--drop-width', help='thickness of streak', default=1, type=int)
    parser.add_arguemnt('-s', '--slant', help='slant of streak', default=0, type=int)
    parser.add_argument('-c', '--color', help='rain streak color', default=200, type=int)
    parser.add_argument('-rc', '--rain-count', help='rain drop count', default=20, type=int)
    

    return parser.parse_args()

# helper functions

In [2]:
def motion_blur(blur_img, kernel_size=30):
    # Specify the kernel size.
    # The greater the size, the more the motion.
    kernel_size = kernel_size

    # Create the vertical kernel.
    kernel_v = np.zeros((kernel_size, kernel_size))

    # Fill the middle row with ones.
    kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)

    # Normalize.
    kernel_v /= kernel_size

    # Apply the vertical kernel.
    vertical_mb = cv2.filter2D(blur_img, -1, kernel_v)
    
    return vertical_mb

In [3]:
def rotate(img, deg=33):
    rotated = imutils.rotate_bound(img, deg)
    return rotated

In [4]:
def rotate_and_blur(img, deg=33, blur_kernel=30):
    x = img.shape[0]
    y = img.shape[1]
    
    # rotate blur and rotate back
    img = rotate(img, deg)
    blured_img = motion_blur(img, kernel_size = blur_kernel)
    rotated = rotate(blured_img, deg)
    
    # crop
    x_hat = rotated.shape[0]
    y_hat = rotated.shape[1]
    
    dx = x_hat - x
    dy = y_hat - y
    
    if dx % 2 == 1:
        x_hat -= 1
        dx -= 1
    if dy % 2 == 1:
        y_hat -= 1
        dy -= 1
        
#     print(x, y)
#     print(x_hat, y_hat)
    
    # shape = (x, y)
    rotated = rotated[dx//2 : x_hat - (dx//2), dy//2 : y_hat - (dy//2)]
    
    # shape = (x, y, 1)
    rotated = np.expand_dims(rotated[:,:,0], axis=2)
    return rotated

In [5]:
def generate_random_lines(imshape,slant,drop_length, rain_count):
    drops=[]
    
    ## If You want heavy rain, try increasing rain_count
    for i in range(rain_count):
        if slant<0:
            x = np.random.randint(slant,imshape[1])
            y = np.random.randint(drop_length, imshape[0])
        else:
            x = np.random.randint(0,imshape[1]-slant)
            y = np.random.randint(0,imshape[0]-drop_length)

        drops.append((x,y))
    return drops

# for drop_color
c = 200

def add_rain(image, drop_length=30, slant=0, drop_width=1, drop_color=(c,c,c), rain_count=50):
    '''
    PARAMS:
    image - input image
    drop_length - how long rain streaks are
    slant - slant angle
    drop_width - rain streak thickness
    drop_color - color
    rain_count - number of droplets
    '''
    imshape = image.shape
    rain_streak_img = np.zeros((imshape[0], imshape[1],imshape[2]))
    
    rain_drops = generate_random_lines(imshape,slant,drop_length, rain_count)
    for rain_drop in rain_drops:
        cv2.line(rain_streak_img,(rain_drop[0],rain_drop[1]),(rain_drop[0]+slant,rain_drop[1]+drop_length),drop_color,drop_width)
        
    rain_streak_img = rotate_and_blur(rain_streak_img, blur_kernel=30)
    return rain_streak_img

In [18]:
img_path = 'rain.jpg'
img = cv2.imread(f"{img_path}")
img.shape

(430, 300, 3)

In [24]:
rainy_img = add_rain(img)
cv2.imwrite(f'./img_n{1}_rc{1}_{1}.jpg', rainy_img)
cv2.imwrite(f'./img_n{1}_rc{1}_{2}.jpg', img+rainy_img)

True

True

In [6]:

images = glob.glob('./all-dogs/*')

In [7]:
# train
for idx, image in enumerate(images[:int(len(images) * 0.6 )]):
    img = cv2.imread(f"{image}")
    
    # 5 rain types
    # 100 per type = 500 rainy images per clean image
    for rain_count in range(20, 101, 10):
        for i in range(100):
            rainy_img = add_rain(img, rain_count=rain_count)
            cv2.imwrite(f'train/img_n{idx}_rc{rain_count}_{i}.jpg',  img + rainy_img)
            

In [8]:
# validation
for idx, image in enumerate(images[int(len(images) * 0.6) : int(len(images) * 0.8)]):
    img = cv2.imread(f"{image}")
    idx += 12347 # for naming sake
    
    # 5 rain types
    # 100 per type = 500 rainy images per clean image
    for rain_count in range(20, 101, 10):
        for i in range(100):
            rainy_img = add_rain(img, rain_count=rain_count)
            cv2.imwrite(f'validation/img_n{idx}_rc{rain_count}_{i}.jpg', img + rainy_img)


In [9]:
# test
for idx, image in enumerate(images[int(len(images) * 0.8): ]):
    img = cv2.imread(f"{image}")
    idx = 16463 # for naming sake
    
    # 5 rain types
    # 100 per type = 500 rainy images per clean image
    for rain_count in range(20, 101, 10):
        for i in range(100):
            rainy_img = add_rain(img, rain_count=rain_count)
            cv2.imwrite(f'test/img_n{idx}_rc{rain_count}_{i}.jpg', img + rainy_img)
            